### Dreambooth Fine-tuning
DreamBooth is a deep learning generation model used to fine-tune existing text-to-image models, developed by researchers from Google Research and Boston University in 2022. Originally developed using Google's own Imagen text-to-image model, DreamBooth implementations can be applied to other text-to-image models, where it can allow the model to generate more fine-tuned and personalised outputs after training on three to five images of a subject.

We should use dreambooth fine tuning our stable diffusion model.

#### Notebook step
1. Import boto3, sagemaker python SDK
2. Build dreambooth fine-tuning image
3. Fine-tuning 
   * config hyperparameter
   * create training job
4. Testing 

#### 1. Import boto3, sagemaker python SDK

In [ ]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

images_s3uri = 's3://{0}/dreambooth/images/'.format(bucket)
models_s3uri = 's3://{0}/stable-diffusion/models/'.format(bucket)
dreambooth_s3uri = 's3://{0}/stable-diffusion/dreambooth/'.format(bucket)

print(bucket)

#### 2. Build dreambooth fine-tuning image 
  It will take 60~90 minutes if use small notebook instance(ml.t3.xlarge)

In [ ]:
!./build_push.sh

#### 3. Fine-tuning 

   * image_uri: docker image ecr URI
   * instance_type: training job instance , prefere ml.g4dn.xlarge, ml.g5.xlarge
   * class_prompt: class prompt
   * instance_prompt: your image key prompt
   * model_name: pretrained_model 
   

In [ ]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}




image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sd-dreambooth-finetuning-v2'
instance_type = 'ml.g4dn.2xlarge'

instance_prompt="photo\ of\ zwx\  man"
class_prompt="photo\ of\ a\ man"
s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'dreambooth', 'trained_models')
model_name="runwayml/stable-diffusion-v1-5"
instance_dir="/opt/ml/input/data/images/"
class_dir="/opt/ml/input/data/class_images/"



environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'model_name':'aws-trained-dreambooth-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': model_name, 
                    'instance_data_dir':instance_dir,
                    'class_data_dir':class_dir,
                    'with_prior_preservation':True,
                    'models_path': '/opt/ml/model/',
                    'instance_prompt': instance_prompt, 
                    'class_prompt':class_prompt,
                    'resolution':512,
                    'train_batch_size':1,
                    'sample_batch_size': 1,
                    'gradient_accumulation_steps':1,
                    'learning_rate':2e-06,
                    'lr_scheduler':'constant',
                    'lr_warmup_steps':0,
                    'num_class_images':50,
                    'max_train_steps':300,
                    'save_steps':300,
                    'attention':'xformers',
                    'prior_loss_weight': 0.5,
                    'use_ema':True,
                    'train_text_encoder':False,
                    'not_cache_latents':True,
                    'gradient_checkpointing':True,
                    'save_use_epochs': False,
                    'use_8bit_adam': False
}

hyperparameters = json_encode_hyperparameters(hyperparameters)



   * Create training job 

In [ ]:
from sagemaker.estimator import Estimator
inputs = {
    'images': f"s3://sagemaker-{region_name}-{account_id}/dreambooth/images/"
}


estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit(inputs)

In [ ]:
dreambooth_model_data = estimator.model_data
print("Model artifact saved at:\n", dreambooth_model_data)

#### 4. Testing 
  you can use inference notebook load your new model